# TensorFlow model saving and loading models and pipelines in TensorFlow.

This notebook covers comprehensive model saving and loading techniques in TensorFlow/Keras.

## Topics Covered:
1. Overview of Saving Formats
2. Saving Keras Models (Full Model)
3. Saving Only Weights
4. Using TF Checkpoints Manually
5. Saving Custom Models (Subclassed)
6. Saving and Loading Pipelines
7. Save Format Comparison
8. Saving and Loading Optimizers
9. Partial Saving (Single Layer)
10. Best Practices

In [ ]:
%pip install tensorflow keras numpy

In [ ]:
# Import all required libraries
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

# Create assets/models directory if it doesn't exist
os.makedirs("./assets/models", exist_ok=True)

print(f"TensorFlow version: {tf.__version__}")

## 1. Overview of Saving Formats in TensorFlow

TensorFlow supports two primary saving mechanisms:

### SavedModel Format
- The official TF standard
- Portable across languages and platforms
- Includes computation graph, variables, assets, signatures
- **Use when:** Exporting models for deployment or sharing

### Checkpoints
- Stores only weights (and sometimes optimizer state)
- Not portable across frameworks
- **Use when:** During training to resume or recover

### H5 Format (.h5)
- Keras-specific format
- Saves architecture + weights
- Lighter than SavedModel but less portable

## 2. Saving Keras Models (Full Model)

This saves architecture + weights + training configuration + optimizer state.

In [ ]:
# Create a simple model
model = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(4,)),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Create dummy data for training
X_train = np.random.randn(100, 4)
y_train = np.random.randn(100, 1)

# Train the model briefly
model.fit(X_train, y_train, epochs=3, verbose=0)

# Save full model (SavedModel format)
model.save("./assets/models/full_model")
print("✓ Model saved to ./assets/models/full_model")

In [ ]:
# Load the full model
loaded_model = keras.models.load_model("./assets/models/full_model")
print("✓ Model loaded successfully\n")

# Display model summary
loaded_model.summary()

# Test the loaded model
test_data = np.random.randn(5, 4)
predictions = loaded_model.predict(test_data, verbose=0)
print(f"\nTest predictions shape: {predictions.shape}")

## 3. Saving Only Weights

Weights-only saving is faster and uses less disk space. Useful during training checkpoints.

In [ ]:
# Save weights only
model.save_weights("./assets/models/ckpt_weights")
print("✓ Weights saved to ./assets/models/ckpt_weights")

# Create a new model with the same architecture
model2 = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(4,)),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
])

# Load the weights into the new model
model2.load_weights("./assets/models/ckpt_weights")
print("✓ Weights loaded into new model")

# Verify both models give same predictions
test_input = np.random.randn(1, 4)
pred1 = model.predict(test_input, verbose=0)
pred2 = model2.predict(test_input, verbose=0)
print(f"\nOriginal model prediction: {pred1[0][0]:.6f}")
print(f"New model prediction: {pred2[0][0]:.6f}")
print(f"Match: {np.allclose(pred1, pred2)}")

## 4. Using TF Checkpoints Manually

Useful for custom training loops or non-Keras models. Allows saving arbitrary variables.

In [ ]:
# Create some variables
v1 = tf.Variable(3.0, name="var1")
v2 = tf.Variable(4.0, name="var2")
step = tf.Variable(1, name="step")

# Create checkpoint
ckpt = tf.train.Checkpoint(step=step, val1=v1, val2=v2)
save_path = ckpt.save("./assets/models/tf_ckpt")
print(f"✓ Checkpoint saved to {save_path}")

# Create new variables
v1_new = tf.Variable(0.0, name="var1_new")
v2_new = tf.Variable(0.0, name="var2_new")
step_new = tf.Variable(0, name="step_new")

# Restore checkpoint
ckpt2 = tf.train.Checkpoint(step=step_new, val1=v1_new, val2=v2_new)
ckpt2.restore(tf.train.latest_checkpoint("./assets/models"))
print("✓ Checkpoint restored")

print(f"\nRestored values:")
print(f"  step: {ckpt2.step.numpy()}")
print(f"  val1: {ckpt2.val1.numpy()}")
print(f"  val2: {ckpt2.val2.numpy()}")

## 5. Saving Custom Models (Subclassed)

Subclassed models can be saved using SavedModel format. The model must be built before saving.

In [ ]:
# Define a custom subclassed model
class MyCustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = keras.layers.Dense(8, activation="relu")
        self.d2 = keras.layers.Dense(4, activation="relu")
        self.d3 = keras.layers.Dense(1)

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

# Create and build the model
custom_model = MyCustomModel()
custom_model(tf.ones((1, 4)))  # Build the model by calling it
custom_model.compile(optimizer="adam", loss="mse")

# Train briefly
custom_model.fit(X_train, y_train, epochs=2, verbose=0)

# Save the custom model
custom_model.save("./assets/models/subclass_savedmodel")
print("✓ Custom model saved to ./assets/models/subclass_savedmodel")

In [ ]:
# Load the custom model
loaded_custom = keras.models.load_model("./assets/models/subclass_savedmodel")
print("✓ Custom model loaded successfully")

# Test predictions
test_pred = loaded_custom.predict(np.random.randn(3, 4), verbose=0)
print(f"\nPredictions shape: {test_pred.shape}")
print(f"Sample prediction: {test_pred[0][0]:.6f}")

## 6. Saving and Loading Pipelines

Preprocessing layers can be saved within the model to ensure consistent input processing.

In [ ]:
# Create a text processing model with preprocessing
text_model = keras.Sequential([
    keras.layers.TextVectorization(max_tokens=1000, output_sequence_length=20),
    keras.layers.Embedding(1000, 16),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

# Adapt the TextVectorization layer to some sample text
sample_texts = [
    "hello world",
    "tensorflow saving pipeline",
    "machine learning model",
    "deep learning framework",
    "neural network training"
]
text_model.layers[0].adapt(sample_texts)

# Compile and build the model
text_model.compile(optimizer="adam", loss="binary_crossentropy")

# Save the entire pipeline
text_model.save("./assets/models/nlp_pipeline_model")
print("✓ NLP pipeline model saved to ./assets/models/nlp_pipeline_model")

In [ ]:
# Load the pipeline model
loaded_pipeline = keras.models.load_model("./assets/models/nlp_pipeline_model")
print("✓ NLP pipeline model loaded successfully")

# Test with new text (preprocessing is included!)
test_texts = ["hello world", "new tensorflow model"]
predictions = loaded_pipeline.predict(test_texts, verbose=0)
print(f"\nPredictions for test texts:")
for text, pred in zip(test_texts, predictions):
    print(f"  '{text}': {pred[0]:.6f}")

## 7. Save Format Comparison

| Format | What it saves | Pros | Cons | Use when |
|--------|---------------|------|------|----------|
| **SavedModel** | Graph, weights, signatures | Standard, portable, deployable | Larger size | Deployment, sharing |
| **H5 (.h5)** | Keras architecture + weights | Lightweight, readable | Not all TF features supported | Classic Keras models |
| **Checkpoints** | Weights only | Fast, minimal storage | No architecture | Training recovery |
| **Custom Objects** | Preprocessing layers | Reproducible inputs | May need adaptation | Consistent preprocessing |

## 7.5 SavedModel vs Checkpoint (Mental Model)

| Need | Use |
|------|-----|
| Resume training exactly | Checkpoint |
| Deploy for inference | SavedModel |
| Share with others | SavedModel |
| Save full computation graph | SavedModel |
| Save intermediate states during long training | Checkpoint |
| Save tokenizer assets with the model | SavedModel |

**Note:** One format cannot replace the other.


## 8. Saving and Loading Optimizers

When saving a compiled model, the optimizer state is also preserved.

In [ ]:
# Create and compile a model with a specific optimizer
opt_model = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(4,)),
    keras.layers.Dense(1)
])

opt_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=["mae"]
)

# Train the model
opt_model.fit(X_train, y_train, epochs=3, verbose=0)

# Save the model (includes optimizer state)
opt_model.save("./assets/models/model_plus_opt")
print("✓ Model with optimizer saved")

# Load the model
loaded_opt_model = keras.models.load_model("./assets/models/model_plus_opt")
print("✓ Model with optimizer loaded")

# Check optimizer
print(f"\nOptimizer type: {type(loaded_opt_model.optimizer).__name__}")
print(f"Learning rate: {loaded_opt_model.optimizer.learning_rate.numpy()}")

## 9. Partial Saving (Single Layer)

You can save and load individual layers, useful for transfer learning or layer reuse.

In [ ]:
# Create a single layer
single_layer = keras.layers.Dense(4, activation="relu", input_shape=(8,))

# Build the layer
test_input = tf.ones((1, 8))
single_layer(test_input)

# Save layer weights
single_layer.save_weights("./assets/models/dense_layer_weights")
print("✓ Layer weights saved")

# Create a new layer with the same architecture
new_layer = keras.layers.Dense(4, activation="relu", input_shape=(8,))
new_layer(test_input)  # Build it first

# Load the weights
new_layer.load_weights("./assets/models/dense_layer_weights")
print("✓ Layer weights loaded")

# Verify they produce the same output
output1 = single_layer(test_input)
output2 = new_layer(test_input)
print(f"\nOutputs match: {np.allclose(output1, output2)}")

## 10. Best Practices

### Key Recommendations:

1. **Use SavedModel for deployment** - It's the standard format for production and cross-platform compatibility

2. **Use checkpoints during training** - Lightweight and fast for saving progress during long training runs

3. **Include preprocessing in the model** - Save preprocessing layers with the model to ensure consistent input handling

4. **Always test loaded models** - Verify predictions match expected behavior after loading

5. **Version your models** - Use descriptive names or version numbers to track different model iterations

6. **Save optimizer state for resuming training** - Use full model save if you plan to continue training later

7. **Consider storage space** - SavedModel is larger but more complete; use weights-only for space efficiency

8. **Document model requirements** - Keep track of TensorFlow version and custom objects used

In [ ]:
# Summary demonstration: Save and load comparison
print("=" * 60)
print("TENSORFLOW MODEL SAVING & LOADING SUMMARY")
print("=" * 60)

# Check what was saved
import glob

saved_items = {
    "Full Model (SavedModel)": "./assets/models/full_model",
    "Weights Only": "./assets/models/ckpt_weights*",
    "TF Checkpoint": "./assets/models/tf_ckpt*",
    "Custom Model": "./assets/models/subclass_savedmodel",
    "NLP Pipeline": "./assets/models/nlp_pipeline_model",
    "Model + Optimizer": "./assets/models/model_plus_opt",
    "Single Layer": "./assets/models/dense_layer_weights*"
}

print("\nSaved artifacts:")
for name, pattern in saved_items.items():
    files = glob.glob(pattern)
    if files:
        print(f"  ✓ {name}")
    else:
        print(f"  ✗ {name}")

print("\n" + "=" * 60)
print("All sections completed successfully!")
print("=" * 60)

# TensorFlow Model Saving and Loading
code example

## 1. SavedModel Format

In [ ]:
import tensorflow as tf
import numpy as np
import os

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')

x = np.random.rand(100, 5)
y = np.random.rand(100, 1)
model.fit(x, y, epochs=1, verbose=0)

save_path = './assets/models/saved_model'
os.makedirs('./assets/models', exist_ok=True)
model.save(save_path)

loaded = tf.keras.models.load_model(save_path)
print("Loaded model:", loaded)

## 2. H5 Format

In [ ]:
h5_path = './assets/models/model.h5'
model.save(h5_path)
loaded_h5 = tf.keras.models.load_model(h5_path)
print(loaded_h5)

## 3. Checkpoints

In [ ]:
ckpt_dir = './assets/models/checkpoints'
os.makedirs(ckpt_dir, exist_ok=True)
ckpt_prefix = os.path.join(ckpt_dir, 'ckpt')

ckpt = tf.train.Checkpoint(model=model)
ckpt.save(ckpt_prefix)

new_model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])
new_model.compile(optimizer='adam', loss='mse')

ckpt_rest = tf.train.Checkpoint(model=new_model)
ckpt_rest.restore(tf.train.latest_checkpoint(ckpt_dir)).expect_partial()
print("Restored weights")

## 4. Pipeline Saving Example

In [ ]:
text_vec = tf.keras.layers.TextVectorization(output_mode='int', output_sequence_length=10)
text_vec.adapt(["hello world", "tensorflow saving"])

input_text = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vec(input_text)
x = tf.keras.layers.Embedding(1000, 8)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output = tf.keras.layers.Dense(1)(x)
pipeline_model = tf.keras.Model(input_text, output)

pipeline_model.compile(optimizer='adam', loss='mse')
pipeline_model.fit(["hello world"] * 4, [0.5] * 4, epochs=1, verbose=0)

pipeline_path = './assets/models/pipeline_saved'
pipeline_model.save(pipeline_path)

loaded_pipeline = tf.keras.models.load_model(pipeline_path)
print(loaded_pipeline.predict(["tensorflow saving"]))